## SDGs 

This script takes the full works data pulled earlier, and uses it to analyze the SDG status of SFU publications

In [43]:
import pandas as pd
import numpy as np
import json
import ast

pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', None)

all_sfu_works = pd.read_csv('../data_pulls/sfu_all_works.csv')


/var/folders/1b/f44cqlr17x1fxp5hhksnmkrw0000gp/T/ipykernel_13946/4293264200.py:9: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  all_sfu_works = pd.read_csv('../data_pulls/sfu_all_works.csv')


1. import the full list of all SFU publications

In [44]:
all_sfu_works

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,indexed_in,open_access,authorships,institutions,countries_distinct_count,institutions_distinct_count,corresponding_author_ids,corresponding_institution_ids,apc_list,apc_paid,fwci,cited_by_count,citation_normalized_percentile,cited_by_percentile_year,biblio,is_retracted,is_paratext,is_xpac,primary_topic,topics,keywords,concepts,mesh,locations_count,locations,best_oa_location,sustainable_development_goals,grants,awards,funders,has_content,referenced_works_count,referenced_works,related_works,abstract_inverted_index,counts_by_year,updated_date,created_date
0,https://openalex.org/W2140190241,https://doi.org/10.5860/choice.49-3305,Data mining: concepts and techniques,Data mining: concepts and techniques,2012.0000,2012-02-01,{'openalex': 'https://openalex.org/W2140190241...,en,"{'id': 'doi:10.5860/choice.49-3305', 'is_oa': ...",article,['crossref'],"{'is_oa': False, 'oa_status': 'closed', 'oa_ur...","[{'author_position': 'first', 'author': {'id':...",[],2,3,[],[],NaN,NaN,1715.5796,28848,"{'value': 1.0, 'is_in_top_1_percent': True, 'i...","{'min': 99, 'max': 100}","{'volume': '49', 'issue': '06', 'first_page': ...",False,False,False,"{'id': 'https://openalex.org/T11106', 'display...","[{'id': 'https://openalex.org/T11106', 'displa...",[{'id': 'https://openalex.org/keywords/compute...,"[{'id': 'https://openalex.org/C41008148', 'wik...",[],10,"[{'id': 'doi:10.5860/choice.49-3305', 'is_oa':...",NaN,"[{'score': 0.49000000953674316, 'id': 'https:/...",[],[],[],"{'pdf': False, 'grobid_xml': False}",487,"['https://openalex.org/W589058777', 'https://o...","['https://openalex.org/W2899084033', 'https://...","{'The': [0], 'increasing': [1], 'volume': [2],...","[{'year': 2025, 'cited_by_count': 320}, {'year...",2025-11-06T03:46:38.306776,2025-10-10T00:00:00
1,https://openalex.org/W4288079944,https://doi.org/10.1051/0004-6361/201833910,<i>Planck</i>2018 results,<i>Planck</i>2018 results,2020.0000,2020-04-03,{'openalex': 'https://openalex.org/W4288079944...,en,"{'id': 'doi:10.1051/0004-6361/201833910', 'is_...",article,['crossref'],"{'is_oa': True, 'oa_status': 'hybrid', 'oa_url...","[{'author_position': 'first', 'author': {'id':...",[],21,181,[],[],NaN,NaN,1016.3574,12397,"{'value': 1.0, 'is_in_top_1_percent': True, 'i...","{'min': 90, 'max': 100}","{'volume': '641', 'issue': None, 'first_page':...",False,False,False,"{'id': 'https://openalex.org/T10095', 'display...","[{'id': 'https://openalex.org/T10095', 'displa...",[{'id': 'https://openalex.org/keywords/physics...,"[{'id': 'https://openalex.org/C121332964', 'wi...",[],21,"[{'id': 'doi:10.1051/0004-6361/201833910', 'is...","{'id': 'doi:10.1051/0004-6361/201833910', 'is_...",[],[],[],[],"{'pdf': True, 'grobid_xml': True}",528,"['https://openalex.org/W2122469395', 'https://...","['https://openalex.org/W2047028841', 'https://...","{'We': [0, 90, 264, 432], 'present': [1], 'cos...","[{'year': 2025, 'cited_by_count': 2520}, {'yea...",2025-11-06T03:46:38.306776,2022-07-28T00:00:00
2,https://openalex.org/W2107140090,https://doi.org/10.1017/s1464793105006950,"Freshwater biodiversity: importance, threats, ...","Freshwater biodiversity: importance, threats, ...",2005.0000,2005-12-12,{'openalex': 'https://openalex.org/W2107140090...,en,"{'id': 'doi:10.1017/s1464793105006950', 'is_oa...",review,"['crossref', 'pubmed']","{'is_oa': False, 'oa_status': 'closed', 'oa_ur...","[{'author_position': 'first', 'author': {'id':...",[],8,11,[],[],"{'value': 4890, 'currency': 'USD', 'value_usd'...",NaN,44.9596,7828,"{'value': 0.99965391, 'is_in_top_1_percent': T...","{'min': 99, 'max': 100}","{'volume': '81', 'issue': '2', 'first_page': '...",False,False,False,"{'id': 'https://openalex.org/T10302', 'display...","[{'id': 'https://openalex.org/T10302', 'displa...",[{'id': 'https://openalex.org/keywords/biodive...,"[{'id': 'https://openalex.org/C130217890', 'wi...","[{'descriptor_ui': 'D000818', 'descriptor_name...",7,"[{'id'

2. import the cleaned list of SFU publications, but drop the two SDG cols because we are going to get more detailed SDG info from the full list and append it to the cleaned list later. 

In [45]:
sfu_works_cleaned = pd.DataFrame(pd.read_csv('../data_pulls/sfu_works_cleaned.csv')).drop(columns=['Top Matching SDG', 'SDG Name'])
sfu_works_cleaned

/var/folders/1b/f44cqlr17x1fxp5hhksnmkrw0000gp/T/ipykernel_13946/3926343954.py:1: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  sfu_works_cleaned = pd.DataFrame(pd.read_csv('../data_pulls/sfu_works_cleaned.csv')).drop(columns=['Top Matching SDG', 'SDG Name'])


,OpenAlex ID,Title,Year,FWCI,Citations Received,Citation Percentile,Top 1% Cited,Top 10% Cited,Topic,Subfield,Field,Domain,Language,Publication Type,Is Retracted,Is Paratext,Works Referenced,Collab. Countries,Collab. Institutions,Author Count,First Author,First Author From SFU,Is Open Access,Open Access Status,source_dict,Source Name,Source Type,Collaboration Status,Citation Percentile Category,Has SDG
0,W2140190241,Data mining: concepts and techniques,2012,1715.5796,28848,100.0000,True,True,Data Management and Algorithms,Signal Processing,Computer Science,Physical Sciences,en,article,False,False,487,2,3,3,Jiawei Han,False,False,closed,"{'id': 'https://openalex.org/S2764375719', 'di...",Choice Reviews Online,journal,Int'l Collab.,Top 1%,True
1,W4288079944,<i>Planck</i>2018 results,2020,1016.3574,12397,100.0000,True,True,Cosmology and Gravitation Theories,Astronomy and Astrophysics,Physics and Astronomy,Physical Sciences,en,article,False,False,528,21,181,100+,N. Aghanim,False,True,hybrid,"{'id': 'https://openalex.org/S205231332', 'dis...",Astronomy and Astrophysics,journal,Int'l Collab.,Top 1%,False
2,W2107140090,"Freshwater biodiversity: importance, threats, ...",2005,44.9596,7828,99.9654,True,True,Fish Ecology and Management Studies,Nature and Landscape Conservation,Environmental Science,Physical Sciences,en,review,False,False,186,8,11,11,David Dudgeon,False,False,closed,"{'id': 'https://openalex.org/S86137544', 'disp...",Biological reviews/Biological reviews of the C...,journal,Int'l Collab.,Top 1%,True
3,W2064853889,Mining frequent patterns without candidate gen...,2000,283.2665,6279,100.0000,True,True,Data Mining Algorithms and Applications,Information Systems,Computer Science,Physical Sciences,en,article,False,False,18,1,3,3,Jiawei Han,True,False,closed,"{'id': 'https://openalex.org/S47508943', 'disp...",ACM SIGMOD Record,journal,Canadian Collab.,Top 1%,False
4,W2141718064,"Evolution, Weighting, and Phylogenetic Utility...",1994,13.1724,6068,99.2826,True,True,Genomics and Phylogenetic Studies,Molecular Biology,"Biochemistry, Genetics and Molecular Biology",Life Sciences,en,article,False,False,0,2,6,6,Chris Simon,False,False,closed,"{'id': 'https://openalex.org/S79992667', 'disp...",Annals of the Entomological Society of America,journal,Int'l Collab.,Top 1%,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76147,W980228898,Modelling the Healthy Premature Infant Brain,2013,0.0000,0,2.9930,False,False,Neonatal and fetal brain pathology,"Pediatrics, Perinatology and Child Health",Medicine,Health Sciences,en,article,False,False,0,1,5,5,Brian G. Booth,True,False,closed,NaN,NaN,NaN,Canadian Collab.,Not in top 10%,False
76148,W99633038,Towards Language-Competent Web Search,2010,0.0000,0,2.4781,False,False,Natural Language Processing Techniques,Artificial Intelligence,Computer Science,Physical Sciences,en,book-chapter,False,False,0,1,1,1,Verónica Dahl,True,False,closed,"{'id': 'https://openalex.org/S4210201731', 'di...",Frontiers in artificial intelligence and appli...,journal,SFU Only,Not in top 10%,True
76149,W997702272,Bibliography,2001,0.0000,0,9.5601,False,False,Historical Economic and Social Studies,Economics and Econometrics,"Economics, Econometrics and Finance",Social Sciences,en,book-chapter,False,False,236,1,1,1,Mark Jaccard,True,False,closed,"{'id': 'https://openalex.org/S4306462995', 'di...",Cambridge University Press eBooks,ebook platform,SFU Only,Not in top 10%,True
76150,W99806901,Parsing with Commitment,1989,0.0000,0,5.0930,False,False,"Logic, Reasoning, and Knowledge",Artificial Intelligence,Computer Science,Physical Sciences,en,book-chapter,False,False,0,2,2,2,Harvey Abramson,False,False,closed,NaN,NaN,NaN,Int'l Collab.,Not in top 10%,False


3. clean up and append the SDG data from the full works df to the clean works df

In [46]:
all_sfu_works['sustainable_development_goals'] = all_sfu_works['sustainable_development_goals'].apply(lambda x: ast.literal_eval(x))
all_sfu_works['sustainable_development_goals'][0]

[{'score': 0.49000000953674316,
  'id': 'https://metadata.un.org/sdg/9',
  'display_name': 'Industry, innovation and infrastructure'}]

In [47]:
all_sfu_works['Number of SDGs'] = all_sfu_works['sustainable_development_goals'].apply(lambda x: len(x))
all_sfu_works['Number of SDGs'].unique()

array([1, 0, 2])

the OpenAlex algorithm for assigning SDGs topublications uses an mBERT machine learning model, which calculates the 'score' of each work for each SDG, where the score is the model's predicted probability that a particular work will be relevant for a particular SDG. OpenAlex includes all SDGs with a score greater than 0.4 in the SDG column. For now, it appears that no SFU publication has more than 2 SDGs assigned to it, so I will design the data frame to be able to accomodate up to 2 SDGs. 

Given the fact that OpenAlex does not use the SDG queries from SciVal/Scopus, we cannot expect that the SDGs for works in OpenAlex line up perfectly with those in SciVal/Scopus. 


In [48]:
all_sfu_works

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,indexed_in,open_access,authorships,institutions,countries_distinct_count,institutions_distinct_count,corresponding_author_ids,corresponding_institution_ids,apc_list,apc_paid,fwci,cited_by_count,citation_normalized_percentile,cited_by_percentile_year,biblio,is_retracted,is_paratext,is_xpac,primary_topic,topics,keywords,concepts,mesh,locations_count,locations,best_oa_location,sustainable_development_goals,grants,awards,funders,has_content,referenced_works_count,referenced_works,related_works,abstract_inverted_index,counts_by_year,updated_date,created_date,Number of SDGs
0,https://openalex.org/W2140190241,https://doi.org/10.5860/choice.49-3305,Data mining: concepts and techniques,Data mining: concepts and techniques,2012.0000,2012-02-01,{'openalex': 'https://openalex.org/W2140190241...,en,"{'id': 'doi:10.5860/choice.49-3305', 'is_oa': ...",article,['crossref'],"{'is_oa': False, 'oa_status': 'closed', 'oa_ur...","[{'author_position': 'first', 'author': {'id':...",[],2,3,[],[],NaN,NaN,1715.5796,28848,"{'value': 1.0, 'is_in_top_1_percent': True, 'i...","{'min': 99, 'max': 100}","{'volume': '49', 'issue': '06', 'first_page': ...",False,False,False,"{'id': 'https://openalex.org/T11106', 'display...","[{'id': 'https://openalex.org/T11106', 'displa...",[{'id': 'https://openalex.org/keywords/compute...,"[{'id': 'https://openalex.org/C41008148', 'wik...",[],10,"[{'id': 'doi:10.5860/choice.49-3305', 'is_oa':...",NaN,"[{'score': 0.49000000953674316, 'id': 'https:/...",[],[],[],"{'pdf': False, 'grobid_xml': False}",487,"['https://openalex.org/W589058777', 'https://o...","['https://openalex.org/W2899084033', 'https://...","{'The': [0], 'increasing': [1], 'volume': [2],...","[{'year': 2025, 'cited_by_count': 320}, {'year...",2025-11-06T03:46:38.306776,2025-10-10T00:00:00,1
1,https://openalex.org/W4288079944,https://doi.org/10.1051/0004-6361/201833910,<i>Planck</i>2018 results,<i>Planck</i>2018 results,2020.0000,2020-04-03,{'openalex': 'https://openalex.org/W4288079944...,en,"{'id': 'doi:10.1051/0004-6361/201833910', 'is_...",article,['crossref'],"{'is_oa': True, 'oa_status': 'hybrid', 'oa_url...","[{'author_position': 'first', 'author': {'id':...",[],21,181,[],[],NaN,NaN,1016.3574,12397,"{'value': 1.0, 'is_in_top_1_percent': True, 'i...","{'min': 90, 'max': 100}","{'volume': '641', 'issue': None, 'first_page':...",False,False,False,"{'id': 'https://openalex.org/T10095', 'display...","[{'id': 'https://openalex.org/T10095', 'displa...",[{'id': 'https://openalex.org/keywords/physics...,"[{'id': 'https://openalex.org/C121332964', 'wi...",[],21,"[{'id': 'doi:10.1051/0004-6361/201833910', 'is...","{'id': 'doi:10.1051/0004-6361/201833910', 'is_...",[],[],[],[],"{'pdf': True, 'grobid_xml': True}",528,"['https://openalex.org/W2122469395', 'https://...","['https://openalex.org/W2047028841', 'https://...","{'We': [0, 90, 264, 432], 'present': [1], 'cos...","[{'year': 2025, 'cited_by_count': 2520}, {'yea...",2025-11-06T03:46:38.306776,2022-07-28T00:00:00,0
2,https://openalex.org/W2107140090,https://doi.org/10.1017/s1464793105006950,"Freshwater biodiversity: importance, threats, ...","Freshwater biodiversity: importance, threats, ...",2005.0000,2005-12-12,{'openalex': 'https://openalex.org/W2107140090...,en,"{'id': 'doi:10.1017/s1464793105006950', 'is_oa...",review,"['crossref', 'pubmed']","{'is_oa': False, 'oa_status': 'closed', 'oa_ur...","[{'author_position': 'first', 'author': {'id':...",[],8,11,[],[],"{'value': 4890, 'currency': 'USD', 'value_usd'...",NaN,44.9596,7828,"{'value': 0.99965391, 'is_in_top_1_percent': T...","{'min': 99, 'max': 100}","{'volume': '81', 'issue': '2', 'first_page': '...",False,False,False,"{'id': 'https://openalex.org/T10302', 'display...","[{'id': 'https://openalex.org/T10302', 'displa...",[{'id': 'https://openalex.org/keywords/biodive...,"[{'id': 'https://openalex.org/C130217890', 'wi...","[{'descriptor_ui': 'D000818', 'descriptor

In [49]:
all_sfu_works['sustainable_development_goals']
all_sfu_works['Number of SDGs'].unique()

array([1, 0, 2])

In [50]:
all_sfu_works['Top SDG Number'] = all_sfu_works['sustainable_development_goals'].apply(lambda x: x[0]['id'] if len(x) > 0 else '').apply(lambda y: np.nan if y=='' else y.lstrip('https://metadata.un.org/sdg/')).apply(lambda z: int(z) if type(z)==str else np.nan)
all_sfu_works['Top SDG Name'] = all_sfu_works['sustainable_development_goals'].apply(lambda x: x[0]['display_name'] if len(x) > 0 else '')
all_sfu_works['Next SDG Number'] = all_sfu_works['sustainable_development_goals'].apply(lambda x: x[1]['id'] if len(x) > 1 else '').apply(lambda y: np.nan if y=='' else y.lstrip('https://metadata.un.org/sdg/')).apply(lambda z: int(z) if type(z)==str else np.nan)
all_sfu_works['Next SDG Name'] = all_sfu_works['sustainable_development_goals'].apply(lambda x: x[1]['display_name'] if len(x) > 1 else '')

all_sfu_works

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,indexed_in,open_access,authorships,institutions,countries_distinct_count,institutions_distinct_count,corresponding_author_ids,corresponding_institution_ids,apc_list,apc_paid,fwci,cited_by_count,citation_normalized_percentile,cited_by_percentile_year,biblio,is_retracted,is_paratext,is_xpac,primary_topic,topics,keywords,concepts,mesh,locations_count,locations,best_oa_location,sustainable_development_goals,grants,awards,funders,has_content,referenced_works_count,referenced_works,related_works,abstract_inverted_index,counts_by_year,updated_date,created_date,Number of SDGs,Top SDG Number,Top SDG Name,Next SDG Number,Next SDG Name
0,https://openalex.org/W2140190241,https://doi.org/10.5860/choice.49-3305,Data mining: concepts and techniques,Data mining: concepts and techniques,2012.0000,2012-02-01,{'openalex': 'https://openalex.org/W2140190241...,en,"{'id': 'doi:10.5860/choice.49-3305', 'is_oa': ...",article,['crossref'],"{'is_oa': False, 'oa_status': 'closed', 'oa_ur...","[{'author_position': 'first', 'author': {'id':...",[],2,3,[],[],NaN,NaN,1715.5796,28848,"{'value': 1.0, 'is_in_top_1_percent': True, 'i...","{'min': 99, 'max': 100}","{'volume': '49', 'issue': '06', 'first_page': ...",False,False,False,"{'id': 'https://openalex.org/T11106', 'display...","[{'id': 'https://openalex.org/T11106', 'displa...",[{'id': 'https://openalex.org/keywords/compute...,"[{'id': 'https://openalex.org/C41008148', 'wik...",[],10,"[{'id': 'doi:10.5860/choice.49-3305', 'is_oa':...",NaN,"[{'score': 0.49000000953674316, 'id': 'https:/...",[],[],[],"{'pdf': False, 'grobid_xml': False}",487,"['https://openalex.org/W589058777', 'https://o...","['https://openalex.org/W2899084033', 'https://...","{'The': [0], 'increasing': [1], 'volume': [2],...","[{'year': 2025, 'cited_by_count': 320}, {'year...",2025-11-06T03:46:38.306776,2025-10-10T00:00:00,1,9.0000,"Industry, innovation and infrastructure",NaN,
1,https://openalex.org/W4288079944,https://doi.org/10.1051/0004-6361/201833910,<i>Planck</i>2018 results,<i>Planck</i>2018 results,2020.0000,2020-04-03,{'openalex': 'https://openalex.org/W4288079944...,en,"{'id': 'doi:10.1051/0004-6361/201833910', 'is_...",article,['crossref'],"{'is_oa': True, 'oa_status': 'hybrid', 'oa_url...","[{'author_position': 'first', 'author': {'id':...",[],21,181,[],[],NaN,NaN,1016.3574,12397,"{'value': 1.0, 'is_in_top_1_percent': True, 'i...","{'min': 90, 'max': 100}","{'volume': '641', 'issue': None, 'first_page':...",False,False,False,"{'id': 'https://openalex.org/T10095', 'display...","[{'id': 'https://openalex.org/T10095', 'displa...",[{'id': 'https://openalex.org/keywords/physics...,"[{'id': 'https://openalex.org/C121332964', 'wi...",[],21,"[{'id': 'doi:10.1051/0004-6361/201833910', 'is...","{'id': 'doi:10.1051/0004-6361/201833910', 'is_...",[],[],[],[],"{'pdf': True, 'grobid_xml': True}",528,"['https://openalex.org/W2122469395', 'https://...","['https://openalex.org/W2047028841', 'https://...","{'We': [0, 90, 264, 432], 'present': [1], 'cos...","[{'year': 2025, 'cited_by_count': 2520}, {'yea...",2025-11-06T03:46:38.306776,2022-07-28T00:00:00,0,NaN,,NaN,
2,https://openalex.org/W2107140090,https://doi.org/10.1017/s1464793105006950,"Freshwater biodiversity: importance, threats, ...","Freshwater biodiversity: importance, threats, ...",2005.0000,2005-12-12,{'openalex': 'https://openalex.org/W2107140090...,en,"{'id': 'doi:10.1017/s1464793105006950', 'is_oa...",review,"['crossref', 'pubmed']","{'is_oa': False, 'oa_status': 'closed', 'oa_ur...","[{'author_position': 'first', 'author': {'id':...",[],8,11,[],[],"{'value': 4890, 'currency': 'USD', 'value_usd'...",NaN,44.9596,7828,"{'value': 0.99965391, 'is_in_top_1_percent': T...","{'min': 99, 'max': 100}","{'volume': '81', 'issue': '2', 'first_page': '...",False,False,False,"{'id': 'https://openalex.org/T10302', 'display...","[{'id': 'https://openalex.org/T10302', 'displa...",[{'id': 'https://opena

In [51]:
sdg_merged = sfu_works_cleaned
sdg_merged['Number of SDGs'] = all_sfu_works['Number of SDGs']
sdg_merged['Top SDG Number'] = all_sfu_works['Top SDG Number']
sdg_merged['Top SDG Name'] = all_sfu_works['Top SDG Name']
sdg_merged['Next SDG Number'] = all_sfu_works['Next SDG Number']
sdg_merged['Next SDG Name'] = all_sfu_works['Next SDG Name']
sdg_merged['SDG Number List'] = sdg_merged.apply(lambda x: [float(x['Top SDG Number']), float(x['Next SDG Number'])], axis=1)

type(sdg_merged['SDG Number List'][0][0])

float

#### NOTE: again x is set to sdg_merged literally because it is easier to type and look at 1 letter only 

In [ ]:
x = sdg_merged

In [54]:
sdg_publications_list = x[x['Number of SDGs'] > 0]
sdg_publications_list

,OpenAlex ID,Title,Year,FWCI,Citations Received,Citation Percentile,Top 1% Cited,Top 10% Cited,Topic,Subfield,Field,Domain,Language,Publication Type,Is Retracted,Is Paratext,Works Referenced,Collab. Countries,Collab. Institutions,Author Count,First Author,First Author From SFU,Is Open Access,Open Access Status,source_dict,Source Name,Source Type,Collaboration Status,Citation Percentile Category,Has SDG,Number of SDGs,Top SDG Number,Top SDG Name,Next SDG Number,Next SDG Name,SDG Number List
0,W2140190241,Data mining: concepts and techniques,2012,1715.5796,28848,100.0000,True,True,Data Management and Algorithms,Signal Processing,Computer Science,Physical Sciences,en,article,False,False,487,2,3,3,Jiawei Han,False,False,closed,"{'id': 'https://openalex.org/S2764375719', 'di...",Choice Reviews Online,journal,Int'l Collab.,Top 1%,True,1,9.0000,"Industry, innovation and infrastructure",NaN,,"[9.0, nan]"
2,W2107140090,"Freshwater biodiversity: importance, threats, ...",2005,44.9596,7828,99.9654,True,True,Fish Ecology and Management Studies,Nature and Landscape Conservation,Environmental Science,Physical Sciences,en,review,False,False,186,8,11,11,David Dudgeon,False,False,closed,"{'id': 'https://openalex.org/S86137544', 'disp...",Biological reviews/Biological reviews of the C...,journal,Int'l Collab.,Top 1%,True,1,15.0000,Life in Land,NaN,,"[15.0, nan]"
13,W2115981184,Assimilation of Enterprise Systems: The Effect...,2007,105.0560,3593,100.0000,True,True,ERP Systems Implementation and Impact,Management Information Systems,"Business, Management and Accounting",Social Sciences,en,article,False,False,93,2,4,4,Huigang Liang,False,False,closed,"{'id': 'https://openalex.org/S57293258', 'disp...",MIS Quarterly,journal,Int'l Collab.,Top 1%,True,1,8.0000,Decent work and economic growth,NaN,,"[8.0, nan]"
15,W2163668399,Passive Dynamic Walking,1990,10.6506,3322,98.7430,False,True,Robotic Locomotion and Control,Biomedical Engineering,Engineering,Physical Sciences,en,article,False,False,13,1,1,1,Tad McGeer,True,False,closed,"{'id': 'https://openalex.org/S73484101', 'disp...",The International Journal of Robotics Research,journal,SFU Only,Top 10%,True,1,7.0000,Affordable and clean energy,NaN,,"[7.0, nan]"
16,W2081234327,Development and validation of the Working Alli...,1989,8.4114,3303,98.0036,False,True,Educational and Psychological Assessments,Developmental and Educational Psychology,Psychology,Social Sciences,en,article,False,False,43,1,2,2,Adam O. Horvath,True,False,closed,"{'id': 'https://openalex.org/S135831435', 'dis...",Journal of Counseling Psychology,journal,Canadian Collab.,Top 10%,True,1,17.0000,Partnerships for the goals,NaN,,"[17.0, nan]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76144,W974186048,Cluster Complexes: Electronics and ICT,2009,0.0000,0,12.5159,False,False,Business Strategy and Innovation,Strategy and Management,"Business, Management and Accounting",Social Sciences,en,book-chapter,False,False,0,1,1,1,Brian Wixted,True,False,closed,"{'id': 'https://openalex.org/S4210171077', 'di...",Advances in spatial science,book series,SFU Only,Not in top 10%,False,1,7.0000,Affordable and clean energy,NaN,,"[7.0, nan]"
76146,W980210813,Modifier Structure Grammars,1989,0.0000,0,14.3490,False,False,Natural Language Processing Techniques,Artificial Intelligence,Computer Science,Physical Sciences,en,book-chapter,False,False,0,2,2,2,Harvey Abramson,False,False,closed,NaN,NaN,NaN,Int'l Collab.,Not in top 10%,False,1,14.0000,Life below water,NaN,,"[14.0, nan]"
76147,W980228898,Modelling the Healthy Premature Infant Brain,2013,0.0000,0,2.9930,False,False,Neonatal and fetal brain pathology,"Pediatrics, Perinatology and Child Health",Medicine,Health Sciences,en,article,False,False,0,1,5,5,Brian G. Booth,True,False,closed,NaN,NaN,NaN,Canadian Collab.,Not in top 10%,False,1,2.0000,Zero hunger,NaN,,"[2.0, nan]"
76148,W99633038,Towards Language-Com

In [55]:
sdg_name_number_key = sdg_publications_list[['Top SDG Number', 'Top SDG Name']]
sdg_name_number_key = sdg_name_number_key.rename(columns={'Top SDG Number': 'SDG', 'Top SDG Name': 'Name'})
sdg_name_number_key = sdg_name_number_key.drop_duplicates()
sdg_name_number_key = sdg_name_number_key.sort_values(by='SDG')
sdg_name_number_key['SDG'] = sdg_name_number_key['SDG'].astype(int)
sdg_name_number_key = sdg_name_number_key.reset_index()
sdg_name_number_key = sdg_name_number_key.drop(columns=['index'])

sdg_name_number_key

,SDG,Name
0,1,No poverty
1,2,Zero hunger
2,3,Good health and well-being
3,4,Quality Education
4,5,Gender equality
5,6,Clean water and sanitation
6,7,Affordable and clean energy
7,8,Decent work and economic growth
8,9,"Industry, innovation and infrastructure"
9,10,Reduced inequalities


In [56]:
sdg_publications_grouped = pd.DataFrame(x.groupby('Number of SDGs').count()['OpenAlex ID']).rename(columns={'OpenAlex ID': 'Count'})
sdg_publications_grouped

,Count
Number of SDGs,
0,33456
1,42155
2,541


Since scival/scopus double-counts publications that fall under more than 1 SDG, we will do the same here. 
According to OpenAlex documentation, they include any SDG with a prediction score higher than 0.4 (using their own prediction algorithm, rather than the SDG queries). I am not sure if it is just the nature of the algorithm or a coincidence, but all publications have either 0, 1, or 2 sdgs assigned to them, so I have hard coded in the option for 2 sdgs here, for simplicity, and also because they don't use the scopus queries anyway, so everything is pretty approximate. 

In [57]:
sdg_counter = pd.DataFrame(columns=['Year', 'SDG', 'Number of Publications'])
year_range = list(range(min(x['Year']), max(x['Year'])+1))

sdg_counter['Year'] = year_range * 17
sdg_counter = sdg_counter.sort_values(['Year'])
sdg_counter['SDG'] = list(range(1,18)) * len(year_range)

for year in year_range:
    temp = x[x['Year'] == year]
    for sdg in range(1,18):
        count = 0
        for row in range(len(temp['Year'])):
            if (sdg in x['SDG Number List'][row]):
                count += 1
        sdg_counter.loc[(sdg_counter['Year']==year) & (sdg_counter['SDG']==sdg), 'Number of Publications'] = count
    #print('year ', year, ' complete!')
    
     
sdg_counter = sdg_counter.reset_index().drop(columns=['index'])
sdg_counter = sdg_counter.merge(sdg_name_number_key, on='SDG')
sdg_counter

,Year,SDG,Number of Publications,Name
0,1964,1,0,No poverty
1,1964,2,0,Zero hunger
2,1964,3,0,Good health and well-being
3,1964,4,0,Quality Education
4,1964,5,0,Gender equality
...,...,...,...,...
1049,2025,13,33,Climate action
1050,2025,14,139,Life below water
1051,2025,15,91,Life in Land
1052,2025,16,92,"Peace, Justice and strong institutions"


In [58]:
sum(sdg_counter[sdg_counter['Year']==2021]['Number of Publications'])

2386

In [59]:
x['Top SDG Number'].unique()

array([ 9., nan, 15.,  8.,  7., 17.,  4., 13., 16.,  3., 10., 14., 11.,
        2.,  5.,  6.,  1., 12.])

In [60]:
top_sdgs_by_year_tall = pd.DataFrame(x.groupby(['Year', 'Top SDG Number', 'Top SDG Name']).count()['OpenAlex ID']).rename(columns={'OpenAlex ID': 'Count'}).reset_index()
top_sdgs_by_year_tall['Top SDG Number'] = top_sdgs_by_year_tall['Top SDG Number'].astype(str).apply(lambda y: y.strip('.0'))
top_sdgs_by_year_tall

,Year,Top SDG Number,Top SDG Name,Count
0,1964,16,"Peace, Justice and strong institutions",1
1,1965,8,Decent work and economic growth,1
2,1965,16,"Peace, Justice and strong institutions",1
3,1966,3,Good health and well-being,2
4,1966,4,Quality Education,2
...,...,...,...,...
969,2025,13,Climate action,38
970,2025,14,Life below water,30
971,2025,15,Life in Land,23
972,2025,16,"Peace, Justice and strong institutions",26


In [61]:
top_sdgs_by_year_wide = x[x['Top SDG Number'].isin(range(18))].reset_index()
top_sdgs_by_year_wide['Top SDG Number'] = top_sdgs_by_year_wide['Top SDG Number'].astype(int)
top_sdgs_by_year_wide = pd.DataFrame(top_sdgs_by_year_wide.groupby(['Year', 'Top SDG Number']).count()['OpenAlex ID'].unstack().reset_index())
top_sdgs_by_year_wide['Total'] = top_sdgs_by_year_wide.drop(columns=['Year']).sum(axis=1)
top_sdgs_by_year_wide

Top SDG Number,Year,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,Total
0,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0000,NaN,1.0000
1,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0000,NaN,2.0000
2,1966,NaN,NaN,2.0000,2.0000,1.0000,1.0000,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,3.0000,2.0000,NaN,NaN,12.0000
3,1967,NaN,NaN,NaN,3.0000,NaN,7.0000,1.0000,1.0000,NaN,3.0000,4.0000,NaN,2.0000,2.0000,2.0000,2.0000,NaN,27.0000
4,1968,NaN,1.0000,NaN,2.0000,1.0000,2.0000,8.0000,1.0000,NaN,5.0000,1.0000,NaN,1.0000,6.0000,1.0000,2.0000,NaN,31.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2021,81.0000,48.0000,454.0000,213.0000,122.0000,56.0000,139.0000,67.0000,85.0000,170.0000,113.0000,22.0000,88.0000,234.0000,108.0000,201.0000,66.0000,2267.0000
58,2022,59.0000,62.0000,424.0000,178.0000,165.0000,49.0000,140.0000,70.0000,100.0000,149.0000,100.0000,23.0000,96.0000,193.0000,80.0000,154.0000,45.0000,2087.0000
59,2023,69.0000,66.0000,376.0000,223.0000,165.0000,51.0000,157.0000,62.0000,81.0000,188.0000,98.0000,12.0000,102.0000,204.0000,108.0000,148.0000,64.0000,2174.0000
60,2024,42.0000,101.0000,263.0000,126.0000,165.0000,37.0000,122.0000,48.0000,71.0000,157.0000,74.0000,20.0000,85.0000,131.0000,59.0000,112.0000,37.0000,1650.0000


In [62]:
sdg_publications_list.to_csv('../data_pulls/sdg_publications_list.csv', index=False)
sdg_publications_grouped.to_csv('../data_pulls/sdg_publications_grouped_by_amount.csv')
sdg_counter.to_csv('../data_pulls/sdg_counts_for_works.csv', index=False)
top_sdgs_by_year_tall.to_csv('../data_pulls/top_sdgs_by_year_tall.csv', index=False)
top_sdgs_by_year_wide.to_csv('../data_pulls/top_sdgs_by_year_wide.csv', index=False)